In [2]:
import chess.pgn

def contar_partidas_con_libreria(ruta_archivo):
    contador = 0
    try:
        with open(ruta_archivo, encoding='utf-8') as archivo_pgn:
            while True:
                # Usamos read_headers() en lugar de read_game()
                # Esto es MUCHO más rápido ya que salta la lista de movimientos
                encabezados = chess.pgn.read_headers(archivo_pgn)
                
                if encabezados is None:
                    break
                    
                contador += 1
        return contador
    except FileNotFoundError:
        print("El archivo no fue encontrado.")
        return 0

# Ejemplo de uso
ruta = "lichess_elite_db/lichess_elite_2020-06.pgn"
total = contar_partidas_con_libreria(ruta)
print(f"Total de partidas encontradas: {total}")

Total de partidas encontradas: 424932


In [4]:
import chess.pgn
import os
import glob
from collections import Counter
import statistics

def analizar_directorio_pgn(directorio):
    # Inicializar contadores y listas
    total_partidas = 0
    resultados = Counter()
    aperturas_eco = Counter()
    elos = []
    
    # Buscar todos los archivos .pgn en la carpeta (insensible a mayúsculas/minúsculas)
    archivos_pgn = glob.glob(os.path.join(directorio, "*.pgn"))
    
    print(f"Iniciando análisis de {len(archivos_pgn)} archivos...")
    print("-" * 40)

    for ruta_archivo in archivos_pgn:
        filename = os.path.basename(ruta_archivo)
        print(f"Procesando: {filename}...", end="\r")
        
        try:
            with open(ruta_archivo, encoding='utf-8', errors='ignore') as pgn:
                while True:
                    # Leemos solo los headers para velocidad (saltamos las jugadas)
                    headers = chess.pgn.read_headers(pgn)
                    if headers is None:
                        break
                    
                    total_partidas += 1
                    
                    # 1. Métricas de Resultado
                    result = headers.get("Result", "*")
                    resultados[result] += 1
                    
                    # 2. Métricas de ELO (Ignoramos si no hay ELO o es '?')
                    w_elo = headers.get("WhiteElo")
                    b_elo = headers.get("BlackElo")
                    
                    if w_elo and w_elo.isdigit():
                        elos.append(int(w_elo))
                    if b_elo and b_elo.isdigit():
                        elos.append(int(b_elo))
                        
                    # 3. Métricas de Apertura (ECO Code)
                    eco = headers.get("ECO")
                    if eco:
                        aperturas_eco[eco] += 1
                        
        except Exception as e:
            print(f"\nError leyendo {filename}: {e}")

    print(f"\n{'='*40}")
    print("REPORTE FINAL")
    print(f"{'='*40}")
    
    if total_partidas == 0:
        print("No se encontraron partidas.")
        return

    # Cálculos finales
    victorias_blancas = resultados.get("1-0", 0)
    victorias_negras = resultados.get("0-1", 0)
    tablas = resultados.get("1/2-1/2", 0)
    
    pct_blancas = (victorias_blancas / total_partidas) * 100
    pct_negras = (victorias_negras / total_partidas) * 100
    pct_tablas = (tablas / total_partidas) * 100

    promedio_elo = statistics.mean(elos) if elos else 0
    max_elo = max(elos) if elos else 0
    min_elo = min(elos) if elos else 0

    # Imprimir resultados
    print(f"Total de Partidas:   {total_partidas:,}")
    print("-" * 20)
    print(f"Blancas ganan:       {victorias_blancas:,} ({pct_blancas:.1f}%)")
    print(f"Negras ganan:        {victorias_negras:,} ({pct_negras:.1f}%)")
    print(f"Tablas:              {tablas:,} ({pct_tablas:.1f}%)")
    print("-" * 20)
    print(f"ELO Promedio:        {int(promedio_elo)}")
    print(f"Rango ELO:           {min_elo} - {max_elo}")
    print("-" * 20)
    print("Top 5 Aperturas (ECO):")
    for eco, count in aperturas_eco.most_common(5):
        pct = (count / total_partidas) * 100
        print(f"  {eco}: {count} ({pct:.1f}%)")

# --- CONFIGURACIÓN ---
# Cambia esto por la ruta de tu carpeta
carpeta_con_pgns = "./lichess_elite_db" 

# Ejecutar
if os.path.isdir(carpeta_con_pgns):
    analizar_directorio_pgn(carpeta_con_pgns)
else:
    print(f"El directorio '{carpeta_con_pgns}' no existe.")

Iniciando análisis de 66 archivos...
----------------------------------------
Procesando: lichess_elite_2025-02.pgn...
REPORTE FINAL
Total de Partidas:   26,358,119
--------------------
Blancas ganan:       12,322,578 (46.8%)
Negras ganan:        11,038,271 (41.9%)
Tablas:              2,995,262 (11.4%)
--------------------
ELO Promedio:        2517
Rango ELO:           2200 - 3999
--------------------
Top 5 Aperturas (ECO):
  A45: 741585 (2.8%)
  A00: 675490 (2.6%)
  A40: 610779 (2.3%)
  A04: 534692 (2.0%)
  B06: 521596 (2.0%)
